In [2]:
%load_ext autoreload
%autoreload 1

import pandas as pd
import numpy as np
from collections import Counter
import datetime
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, DayLocator, HourLocator
import gsw
import geomag
import matplotlib.dates as mdates
from cmocean import cm as cmo
import matplotlib as mpl
from tqdm.notebook import tqdm_notebook as tqdm
import sys


sys.path.append('/Volumes/MASSIVEUNIT/Work/SOCHIC_2022/sailbuoy_processing/code/')


%aimport dcps_tools
%aimport dcps_plotting

font = {'family' : 'Avenir',
        'weight' : 'normal',
        'size'   : 30}

mpl.rc('font', **font)

def rot_ticks(axs,rot,ha):
    for xlabels in axs.get_xticklabels():
                xlabels.set_rotation(rot)
                xlabels.set_ha(ha)


mpl.rcParams['xtick.major.size'] = 8
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.minor.size'] = 8
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.size'] = 8
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 8
mpl.rcParams['ytick.minor.width'] = 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
path = '/Volumes/MASSIVEUNIT/Work/SailBuoy Internship/Data/2022_SOCHIC_Full/DCPS.TXT'
desc = 'DCPS from SB Kringla 2022'
ds = dcps_tools.load_dcps(path,desc)

Checking lines in input file:   0%|          | 0/701415 [00:00<?, ?it/s]

14555 good measurements found in input file
67 corrupt measurements found in input file
4 measurements without GPS fix found in input file

ADCP data comprised of 40 cells



Creating dataset:   0%|          | 0/701415 [00:00<?, ?it/s]

Loaded 14555 good measurements to dataset.
Discarded 67 corrupt measurements.
Discarded 4 measurements without GPS fix.


In [6]:
ds

<xarray.Dataset>
Dimensions:                     (time: 14555, cells: 40)
Coordinates:
  * cells                       (cells) int64 0 1 2 3 4 5 ... 34 35 36 37 38 39
  * time                        (time) datetime64[ns] 2022-01-10T12:05:26 ......
Data variables: (12/65)
    Charge Voltage Vtx2[V]      (time) float64 4.974 4.973 4.973 ... 4.969 4.969
    Input Voltage[V]            (time) float64 13.96 13.9 13.86 ... 13.05 13.03
    Heading[Deg.M]              (time) float64 96.34 120.3 112.2 ... 302.8 329.8
    Pitch[Deg]                  (time) float64 0.2601 0.3763 ... 28.93 42.05
    Roll[Deg]                   (time) float64 -179.0 -178.7 ... 152.1 121.5
    Max Tilt[Deg]               (time) float64 3.518 4.153 4.525 ... 90.0 90.0
    ...                          ...
    Vertical Speed[cm/s]        (cells, time) float64 -14.18 0.1991 ... nan nan
    *Beam3 Strength[dB]         (cells, time) float64 nan nan ... -38.86 -15.63
    Horizontal Speed[cm/s]      (cells, time) float64 80.58 46.89 ... nan nan
    *SP Stdev Horizontal[cm/s]  (cells, time) float64 nan nan ... 317.4 130.2
    *Beam3 Speed[cm/s]          (cells, time) float64 nan nan ... 27.61 -8.986
    *Beam2 Stdev[cm/s]          (cells, time) float64 nan nan nan ... 176.9 0.0
Attributes:
    description:  DCPS from SB Kringla 2022

In [7]:
ds = dcps_tools.drop_bad_vars(ds)
ds = dcps_tools.add_attrs(ds)
ds

Dropped 20 variables starting with * from the dataset.
Updated names, units, and description for 45 variables


<xarray.Dataset>
Dimensions:              (time: 14555, cells: 40)
Coordinates:
  * cells                (cells) int64 0 1 2 3 4 5 6 7 ... 33 34 35 36 37 38 39
  * time                 (time) datetime64[ns] 2022-01-10T12:05:26 ... 2022-0...
Data variables: (12/45)
    heading              (time) float64 96.34 120.3 112.2 ... 293.1 302.8 329.8
    sd_heading           (time) float64 1.144 1.49 1.744 ... 23.52 20.79 24.34
    pitch                (time) float64 0.2601 0.3763 0.3317 ... 28.93 42.05
    roll                 (time) float64 -179.0 -178.7 -178.7 ... 152.1 121.5
    abs_tilt             (time) float64 178.6 178.4 178.5 ... 122.4 130.1 103.3
    max_tilt             (time) float64 3.518 4.153 4.525 ... 90.0 90.0 90.0
    ...                   ...
    lat_start            (time) float64 -63.3 -63.34 -63.38 ... -53.7 -53.69
    lat_end              (time) float64 -63.3 -63.35 -63.38 ... -53.7 -53.7
    lon_start            (time) float64 2.448 2.448 2.448 ... 0.1381 0.1167
    lon_end              (time) float64 2.45 2.448 2.45 ... 0.1461 0.1381 0.108
    time_start           (time) datetime64[ns] 2022-01-10T12:05:26 ... 2022-0...
    time_end             (time) datetime64[ns] 2022-01-10T12:09:33 ... 2022-0...
Attributes:
    description:  DCPS from SB Kringla 2022

In [9]:
ds = dcps_tools.calc_heading_speed(ds)
ds = dcps_tools.corr_mag_dec(ds)
ds = dcps_tools.corr_speed(ds)
ds = dcps_tools.add_coords(ds)
ds = dcps_tools.update_attrs(ds)

Corrected for magnetic declination
Corrected for GPS movements
Updated names, units, and description for 9 variables


<xarray.Dataset>
Dimensions:              (time: 14555, cells: 40)
Coordinates:
  * cells                (cells) int64 0 1 2 3 4 5 6 7 ... 33 34 35 36 37 38 39
  * time                 (time) datetime64[ns] 2022-01-10T12:05:26 ... 2022-0...
    depth                (cells) int64 3 5 7 9 11 13 15 ... 69 71 73 75 77 79 81
    latitude             (time) float64 -63.3 -63.34 -63.38 ... -53.7 -53.69
    longitude            (time) float64 2.448 2.448 2.448 ... 0.1381 0.1167
Data variables: (12/54)
    heading              (time) float64 96.34 120.3 112.2 ... 293.1 302.8 329.8
    sd_heading           (time) float64 1.144 1.49 1.744 ... 23.52 20.79 24.34
    pitch                (time) float64 0.2601 0.3763 0.3317 ... 28.93 42.05
    roll                 (time) float64 -179.0 -178.7 -178.7 ... 152.1 121.5
    abs_tilt             (time) float64 178.6 178.4 178.5 ... 122.4 130.1 103.3
    max_tilt             (time) float64 3.518 4.153 4.525 ... 90.0 90.0 90.0
    ...                   ...
    sample_u             (time) float64 0.4349 0.1191 0.2421 ... 0.0 -1.822
    sample_v             (time) float64 -1.648 -5.794 -1.423 ... 0.4334 -6.286
    c_u_mag              (cells, time) float64 -0.536 -0.3223 0.3256 ... nan nan
    c_v_mag              (cells, time) float64 0.6017 0.3407 0.4251 ... nan nan
    c_u_corr             (cells, time) float64 -0.1011 -0.2031 ... nan nan
    c_v_corr             (cells, time) float64 -1.046 -5.453 -0.9975 ... nan nan
Attributes:
    description:  DCPS from SB Kringla 2022

In [10]:
ds

<xarray.Dataset>
Dimensions:              (time: 14555, cells: 40)
Coordinates:
  * cells                (cells) int64 0 1 2 3 4 5 6 7 ... 33 34 35 36 37 38 39
  * time                 (time) datetime64[ns] 2022-01-10T12:05:26 ... 2022-0...
    depth                (cells) int64 3 5 7 9 11 13 15 ... 69 71 73 75 77 79 81
    latitude             (time) float64 -63.3 -63.34 -63.38 ... -53.7 -53.69
    longitude            (time) float64 2.448 2.448 2.448 ... 0.1381 0.1167
Data variables: (12/54)
    heading              (time) float64 96.34 120.3 112.2 ... 293.1 302.8 329.8
    sd_heading           (time) float64 1.144 1.49 1.744 ... 23.52 20.79 24.34
    pitch                (time) float64 0.2601 0.3763 0.3317 ... 28.93 42.05
    roll                 (time) float64 -179.0 -178.7 -178.7 ... 152.1 121.5
    abs_tilt             (time) float64 178.6 178.4 178.5 ... 122.4 130.1 103.3
    max_tilt             (time) float64 3.518 4.153 4.525 ... 90.0 90.0 90.0
    ...                   ...
    sample_u             (time) float64 0.4349 0.1191 0.2421 ... 0.0 -1.822
    sample_v             (time) float64 -1.648 -5.794 -1.423 ... 0.4334 -6.286
    c_u_mag              (cells, time) float64 -0.536 -0.3223 0.3256 ... nan nan
    c_v_mag              (cells, time) float64 0.6017 0.3407 0.4251 ... nan nan
    c_u_corr             (cells, time) float64 -0.1011 -0.2031 ... nan nan
    c_v_corr             (cells, time) float64 -1.046 -5.453 -0.9975 ... nan nan
Attributes:
    description:  DCPS from SB Kringla 2022